In [1]:
!git clone https://github.com/wpeebles/hessian_penalty.git
!git clone https://github.com/Harry24k/adversairal-attacks-pytorch

Cloning into 'hessian_penalty'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 195 (delta 15), reused 91 (delta 15), pack-reused 101
Receiving objects: 100% (195/195), 44.26 MiB | 40.29 MiB/s, done.
Resolving deltas: 100% (52/52), done.
Cloning into 'adversairal-attacks-pytorch'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 1024 (delta 8), reused 7 (delta 1), pack-reused 996
Receiving objects: 100% (1024/1024), 36.31 MiB | 38.21 MiB/s, done.
Resolving deltas: 100% (590/590), done.


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
from torchsummary import summary
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
mnist = tf.keras.datasets.mnist
(X_train,y_train),(X_test,y_test) = mnist.load_data()

import sys
#sys.path.append(r"C:\Users\peter fazekas\Desktop\masters\RNN hessian reg\hessian_penalty-master")


11493376/11490434 [==============================] - 0s 0us/step


In [3]:
from hessian_penalty.hessian_penalty_pytorch import hessian_penalty

In [4]:
X_train = X_train/255
X_test = X_test/255
print("original shape: ",X_train.shape)
X= X_train
print("shape needed for RNN: ",X.shape)
print("ytrains shape",y_train.shape)

original shape:  (60000, 28, 28)
shape needed for RNN:  (60000, 28, 28)
ytrains shape (60000,)


In [132]:
# parameters 
N_STEPS = 28
N_INPUTS = 28
N_NEURONS = 150
N_OUTPUTS = 10
N_EPHOCS = 30
BATCH_SIZE=128

In [133]:


# list all transformations
transform = transforms.Compose([transforms.ToTensor()])

# download and load training dataset
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# download and load testing dataset
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [134]:
# RNN Model
class ImageRNN(nn.Module):
    def __init__(self, batch_size, n_steps, n_inputs, n_neurons, n_outputs):
        super(ImageRNN, self).__init__()
        
        self.n_neurons = n_neurons
        self.batch_size = batch_size
        self.n_steps = n_steps
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.device ="cuda:0"
        
        self.basic_rnn = nn.RNN(self.n_inputs, self.n_neurons) 
        
        self.FC = nn.Linear(self.n_neurons, self.n_outputs)
        
    def init_hidden(self,):
        # (num_layers, batch_size, n_neurons)
        return (torch.zeros(1, self.batch_size, self.n_neurons).to(self.device))
        
    def forward(self, X):
        # transforms X to dimensions: n_steps X batch_size X n_inputs
        X = X.permute(1, 0, 2) 
        
        
        self.batch_size = X.size(1)
        self.hidden = self.init_hidden()
        
        lstm_out, self.hidden = self.basic_rnn(X, self.hidden)      
        out = self.FC(self.hidden)
        
        return out.view(-1, self.n_outputs) # batch_size X n_output
#model = ImageRNN(BATCH_SIZE, N_STEPS, N_INPUTS, N_NEURONS, N_OUTPUTS)


In [135]:
print("GPU is:",torch.cuda.get_device_name(0))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

GPU is: Tesla T4
cuda:0


In [136]:
import torch.optim as optim

# Device
def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()
#reg = hessian_penalty()
lambda_JR = 0.01 # hyperparameter

def train(jacobi,model,N_EPHOCS,lambda_JR):
    # Model instance

    for epoch in range(N_EPHOCS+1):  # loop over the dataset multiple times
        train_running_loss = 0.0
        train_acc = 0.0
        model.train()
        # TRAINING ROUND
        for i, data in enumerate(trainloader,0):
            #only train for a single batch for now since its faster
            if i>=1:
              break
             # zero the parameter gradients
            optimizer.zero_grad()
            # reset hidden states
            model.hidden = model.init_hidden()
            
            # get the inputs
            inputs, labels = data
            if torch.cuda.is_available():
              inputs = inputs.cuda()
              labels = labels.cuda()
            inputs = inputs.view(-1,28, 28) 
            inputs.requires_grad = True # this is essential!

            # forward + backward + optimize
            #print(inputs.shape)
            outputs = model(inputs)
            #print("output shape is ", outputs.shape)
            loss = criterion(outputs, labels)
               # hessian regularization
            if not jacobi:
                total_loss = loss 
                R=hessian_penalty(model,z =inputs).cuda()
            else:
                R = hessian_penalty(model,z =inputs).cuda()
                total_loss = loss +lambda_JR*(R) # full loss

            total_loss.backward()
            optimizer.step()
            
            train_running_loss += loss.detach().item()
            train_acc += get_accuracy(outputs, labels, BATCH_SIZE)
        model.eval()
        #print(i)
        if epoch%10==0:
          print('Epoch:  %d | Loss: %.4f |hessian Loss %.4f | Train Accuracy: %.2f' 
                %(epoch, train_running_loss/i,R, train_acc/i))
        
        
    

In [137]:
models = ImageRNN(BATCH_SIZE, N_STEPS, N_INPUTS, N_NEURONS, N_OUTPUTS).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(models.parameters(), lr=0.001)
model_NH = models.to(device)
if torch.cuda.is_available():
  model_NH =model_NH.cuda()
  criterion=criterion.cuda()

In [138]:
print("training without hessian reg")
N_EPHOCS=300
optimizer = optim.Adam(models.parameters(), lr=0.001)
train(False,model_NH,N_EPHOCS,lambda_JR)


training without hessian reg
Epoch:  0 | Loss: 2.3166 |hessian Loss 0.0031 | Train Accuracy: 7.81
Epoch:  10 | Loss: 1.9951 |hessian Loss 2.8894 | Train Accuracy: 27.34
Epoch:  20 | Loss: 1.5694 |hessian Loss 22394.8105 | Train Accuracy: 45.31
Epoch:  30 | Loss: 1.3036 |hessian Loss 30894.6777 | Train Accuracy: 59.38
Epoch:  40 | Loss: 1.1293 |hessian Loss 118014.1719 | Train Accuracy: 64.84
Epoch:  50 | Loss: 1.0017 |hessian Loss 75535.4844 | Train Accuracy: 68.75
Epoch:  60 | Loss: 0.8831 |hessian Loss 172673.3125 | Train Accuracy: 68.75
Epoch:  70 | Loss: 0.8002 |hessian Loss 92553.3438 | Train Accuracy: 71.88
Epoch:  80 | Loss: 0.7770 |hessian Loss 289685.6875 | Train Accuracy: 69.53
Epoch:  90 | Loss: 0.6458 |hessian Loss 410869.2500 | Train Accuracy: 81.25
Epoch:  100 | Loss: 0.5898 |hessian Loss 270961.8438 | Train Accuracy: 81.25
Epoch:  110 | Loss: 0.5441 |hessian Loss 331847.6562 | Train Accuracy: 83.59
Epoch:  120 | Loss: 0.4908 |hessian Loss 359065.1250 | Train Accuracy: 85

KeyboardInterrupt: ignored

In [139]:
print("training with hessian reg")
#reset the model and train with hessian reg
models = ImageRNN(BATCH_SIZE, N_STEPS, N_INPUTS, N_NEURONS, N_OUTPUTS).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(models.parameters(), lr=0.0001)
model_H = models.to(device)
if torch.cuda.is_available():
  model_H =model_H.cuda()
  criterion=criterion.cuda()


training with hessian reg


In [140]:
#sometimes it gets stuck in local minima so change learning rate, accuracy should be above 80%
optimizer = optim.Adam(models.parameters(), lr=0.001)
train(True,model_H,N_EPHOCS=1000,lambda_JR=0.01)

Epoch:  0 | Loss: 2.3041 |hessian Loss 0.0029 | Train Accuracy: 14.06
Epoch:  10 | Loss: 2.0532 |hessian Loss 0.4981 | Train Accuracy: 36.72
Epoch:  20 | Loss: 1.7431 |hessian Loss 4.6121 | Train Accuracy: 47.66
Epoch:  30 | Loss: 1.5015 |hessian Loss 17.7610 | Train Accuracy: 51.56
Epoch:  40 | Loss: 1.4061 |hessian Loss 4.6988 | Train Accuracy: 56.25
Epoch:  50 | Loss: 1.6545 |hessian Loss 7.8400 | Train Accuracy: 40.62
Epoch:  60 | Loss: 1.4239 |hessian Loss 19.7812 | Train Accuracy: 60.16
Epoch:  70 | Loss: 1.3718 |hessian Loss 6.4652 | Train Accuracy: 62.50
Epoch:  80 | Loss: 1.2517 |hessian Loss 7.1515 | Train Accuracy: 64.06
Epoch:  90 | Loss: 1.2813 |hessian Loss 5.0980 | Train Accuracy: 60.94
Epoch:  100 | Loss: 1.1511 |hessian Loss 23.9883 | Train Accuracy: 64.84
Epoch:  110 | Loss: 1.1914 |hessian Loss 4.6314 | Train Accuracy: 66.41
Epoch:  120 | Loss: 1.0685 |hessian Loss 9.0660 | Train Accuracy: 71.88
Epoch:  130 | Loss: 1.2005 |hessian Loss 13.1593 | Train Accuracy: 62.50

KeyboardInterrupt: ignored

In [141]:
import random
for i, data in enumerate(trainloader,0):
      if i>=1:
        break
      inputs, labels = data
      if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
      print(type(inputs))
      inputs = inputs.view(-1,28, 28)
      accNH = get_accuracy(model_NH(inputs),labels,BATCH_SIZE)
      print("accuracy without hessian on training data",accNH)
      accH = get_accuracy(model_H(inputs),labels,BATCH_SIZE)
      print("accuracy with hessian on training data",accH)
      

<class 'torch.Tensor'>
accuracy without hessian on training data 93.75
accuracy with hessian on training data 82.03125


In [151]:
from adversairal import torchattacks

for i, data in enumerate(trainloader,0):
  if i>=1:
    break
  torch.backends.cudnn.enabled = False
  if torch.cuda.is_available():
        inputs = inputs.cuda()
        labels = labels.cuda()
  inputs, labels = data
  inputs = inputs.view(-1,28, 28) 
  inputs.requires_grad = True # this is essential!
  atk_H = torchattacks.PGD(model_H, eps=10/255, alpha=2/255, steps=3)
  atk_NH = torchattacks.PGD(model_NH, eps=10/255, alpha=2/255, steps=3)
  adversarial_images_H = atk_H(inputs, labels)
  adversarial_images_NH = atk_NH(inputs, labels)
  adversarial_labels = labels
  if torch.cuda.is_available():
        inputs = inputs.cuda()
        labels = labels.cuda()
        adversarial_images = adversarial_images.cuda()
        adversarial_labels = adversarial_labels.cuda()
        model_NH=model_NH.cuda()
        model_H=model_H.cuda()
  accNH = get_accuracy(model_NH(adversarial_images_NH),adversarial_labels,BATCH_SIZE)
  print("accuracy without hessian on adversarial data",accNH)
  print("difference in accuracy without hessian",abs(accNH-get_accuracy(model_NH(inputs),labels,BATCH_SIZE)))
  print("\n")
  accH = get_accuracy(model_H(adversarial_images_H),adversarial_labels,BATCH_SIZE)
  print("accuracy with hessian on adversarial data",accH)
  print("difference in accuracy with hessian",abs(accH-get_accuracy(model_H(inputs),labels,BATCH_SIZE)))
  print("\n")

accuracy without hessian on adversarial data 21.875
difference in accuracy without hessian 71.875


accuracy with hessian on adversarial data 76.5625
difference in accuracy with hessian 5.46875




**Adversarial attacks with random perturb**

In [148]:

from adversairal import torchattacks
global inputs,labels
for i, data in enumerate(trainloader,0):
  #train for 1 batches
  if i>=1:
    break
  torch.backends.cudnn.enabled = False
  if torch.cuda.is_available():
        inputs = inputs.cuda()
        labels = labels.cuda()
  inputs, labels = data
  orig_in,orig_lab=inputs,labels
  inputs = inputs.view(-1,28, 28) 
  orig_in = orig_in.view(-1,28,28)
  inputs.requires_grad = True # this is essential!
  p_inputs = inputs
  p_inputs.requires_grad = False
  p_labels = labels
  
  for k in range(p_inputs.shape[0]):
    #sample set of points and then divide by the norm of it
    #np.random.seed(1)
    vecs = np.random.uniform(-1,1,(1,p_inputs.shape[1],p_inputs.shape[2]))
    vecs/= np.linalg.norm(vecs)
    vecs=0.1*vecs
    #print(vecs)       
    vecs = torch.from_numpy(vecs).cuda()
    p_inputs[k] = torch.add(p_inputs[k].cuda(),vecs)
  inputs =p_inputs.cuda()
  atk_H = torchattacks.PGD(model_H, eps=10/255, alpha=2/255, steps=4)
  atk_NH = torchattacks.PGD(model_NH, eps=10/255, alpha=2/255, steps=4)
  adversarial_images_H = atk_H(inputs, labels)
  adversarial_images_NH = atk_NH(inputs, labels)
  adversarial_labels = labels
  if torch.cuda.is_available():
        inputs = inputs.cuda()
        labels = labels.cuda()
        orig_in=orig_in.cuda()
        orig_lab=orig_lab.cuda()
        adversarial_images = adversarial_images.cuda()
        adversarial_labels = adversarial_labels.cuda()
        model_NH=model_NH.cuda()
        model_H=model_H.cuda()
  accNH = get_accuracy(model_NH(adversarial_images_NH),adversarial_labels,BATCH_SIZE)
  print("accuracy without hessian on adversarial perturbed data",accNH)
  print("difference in accuracy without hessian",abs(accNH-get_accuracy(model_NH(orig_in),labels,BATCH_SIZE)))
  accH = get_accuracy(model_H(adversarial_images_H),adversarial_labels,BATCH_SIZE)
  print("accuracy with hessian on adversarial perturbed data",accH)
  print("difference in accuracy with hessian",abs(accH-get_accuracy(model_H(orig_in),labels,BATCH_SIZE)))
  print("\n")

accuracy without hessian on adversarial perturbed data 13.28125
difference in accuracy without hessian 79.6875
accuracy with hessian on adversarial perturbed data 74.21875
difference in accuracy with hessian 7.8125




**Block below is for perturbing points**

In [149]:
#code to evaluate the first batch
import random
for i, data in enumerate(trainloader,0):
      inputs, labels = data
      if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
      print(type(inputs))
      inputs = inputs.view(-1,28, 28)
      accNH = get_accuracy(model_NH(inputs),labels,BATCH_SIZE)
      print("accuracy without hessian on training data",accNH)
      accH = get_accuracy(model_H(inputs),labels,BATCH_SIZE)
      print("accuracy with hessian on training data",accH)
      break

<class 'torch.Tensor'>
accuracy without hessian on training data 93.75
accuracy with hessian on training data 82.03125


In [175]:
def train_p(jacobi,model,N_EPHOCS,lambda_JR):
    # Model instance
    for epoch in range(N_EPHOCS+1):  # loop over the dataset multiple times
        train_running_loss = 0.0
        train_acc = 0.0
        model.train()
        # TRAINING ROUND
        for i, data in enumerate(trainloader,0):
            #only train for a single batch for now since its faster
            if i>=1:
              break
             # zero the parameter gradients
            optimizer.zero_grad()
            # reset hidden states
            model.hidden = model.init_hidden()
            
            # get the inputs
            inputs, labels = data
            
            if torch.cuda.is_available():
              inputs = inputs.cuda()
              labels = labels.cuda()
            inputs = inputs.view(-1,28, 28) 
            inputs.requires_grad = True # this is essential!
            
            p_inputs = inputs
            p_inputs.requires_grad = False
            p_labels = labels
            
            for k in range(p_inputs.shape[0]):
              #sample set of points and then divide by the norm of it
              #np.random.seed(1)
              vecs = np.random.uniform(-1,1,(1,p_inputs.shape[1],p_inputs.shape[2]))
              #vecs/= np.linalg.norm(vecs)
              #print(np.linalg.norm(vecs))
              vecs=vecs
              #print(vecs)       
              vecs = torch.from_numpy(vecs).cuda()
              p_inputs[k] = torch.add(p_inputs[k],vecs)
              
            p_inputs = torch.cat((p_inputs,inputs),0)
            labels = torch.cat((labels,labels))
            #print(p_inputs.shape)
            outputs = model(p_inputs)
            loss = criterion(outputs, labels)
               # hessian regularization
            if not jacobi:
                total_loss = loss 
                R=0
            else:
                R = hessian_penalty(model,z =p_inputs).cuda()
                total_loss = loss +lambda_JR*(R) # full loss

            total_loss.backward()
            optimizer.step()
            
            train_running_loss += loss.detach().item()
            train_acc += get_accuracy(outputs, labels, 2*BATCH_SIZE)
        model.eval()
        #print(i)
        if epoch%10==0:
          print('Epoch:  %d | Loss: %.4f |hessian Loss %.4f | Train Accuracy: %.2f' 
                %(epoch, train_running_loss/i,R, train_acc/i))

In [176]:
optimizer = optim.Adam(model_H.parameters(), lr=0.001)
train_p(True,model_H,N_EPHOCS=300,lambda_JR=0.05)

Epoch:  0 | Loss: 0.9565 |hessian Loss 3.6228 | Train Accuracy: 66.41
Epoch:  10 | Loss: 1.0154 |hessian Loss 3.0275 | Train Accuracy: 67.19
Epoch:  20 | Loss: 0.8405 |hessian Loss 4.6601 | Train Accuracy: 73.44
Epoch:  30 | Loss: 0.8756 |hessian Loss 3.9355 | Train Accuracy: 75.00
Epoch:  40 | Loss: 0.8651 |hessian Loss 1.8074 | Train Accuracy: 75.78
Epoch:  50 | Loss: 0.7768 |hessian Loss 2.4625 | Train Accuracy: 75.78
Epoch:  60 | Loss: 0.9314 |hessian Loss 3.0850 | Train Accuracy: 68.75
Epoch:  70 | Loss: 0.8355 |hessian Loss 4.2727 | Train Accuracy: 78.12
Epoch:  80 | Loss: 0.7590 |hessian Loss 3.1515 | Train Accuracy: 74.22
Epoch:  90 | Loss: 0.6608 |hessian Loss 4.9627 | Train Accuracy: 85.16


KeyboardInterrupt: ignored

In [ ]:
optimizer = optim.Adam(model_NH.parameters(), lr=0.001)
N_EPHOCS=300
train_p(False,model_NH,N_EPHOCS,lambda_JR)

In [177]:
print("these are the results after random training")
for i, data in enumerate(trainloader,0):
      if i>=1:
        break
      inputs, labels = data
      if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
      print(type(inputs))
      inputs = inputs.view(-1,28, 28)
      accNH = get_accuracy(model_NH(inputs),labels,BATCH_SIZE)
      print("accuracy without hessian on training data",accNH)
      accH = get_accuracy(model_H(inputs),labels,BATCH_SIZE)
      print("accuracy with hessian on training data",accH)
      

these are the results after random training
<class 'torch.Tensor'>
accuracy without hessian on training data 92.1875
accuracy with hessian on training data 82.03125


In [178]:
from adversairal import torchattacks

for i, data in enumerate(trainloader,0):
  if i>=1:
    break
  torch.backends.cudnn.enabled = False
  if torch.cuda.is_available():
        inputs = inputs.cuda()
        labels = labels.cuda()
  inputs, labels = data
  inputs = inputs.view(-1,28, 28) 
  inputs.requires_grad = True # this is essential!
  atk_H = torchattacks.PGD(model_H, eps=10/255, alpha=2/255, steps=3)
  atk_NH = torchattacks.PGD(model_NH, eps=10/255, alpha=2/255, steps=3)
  adversarial_images_H = atk_H(inputs, labels)
  adversarial_images_NH = atk_NH(inputs, labels)
  adversarial_labels = labels
  if torch.cuda.is_available():
        inputs = inputs.cuda()
        labels = labels.cuda()
        adversarial_images_H = adversarial_images_H.cuda()
        adversarial_images_NH = adversarial_images_NH.cuda()
        adversarial_labels = adversarial_labels.cuda()
        model_NH=model_NH.cuda()
        model_H=model_H.cuda()
  accNH = get_accuracy(model_NH(adversarial_images_NH),adversarial_labels,BATCH_SIZE)
  print("accuracy without hessian on adversarial data",accNH)
  print("difference in accuracy without hessian",abs(accNH-get_accuracy(model_NH(inputs),labels,BATCH_SIZE)))
  print("\n")
  accH = get_accuracy(model_H(adversarial_images_H),adversarial_labels,BATCH_SIZE)
  print("accuracy with hessian on adversarial data",accH)
  print("difference in accuracy with hessian",abs(accH-get_accuracy(model_H(inputs),labels,BATCH_SIZE)))
  print("\n")

accuracy without hessian on adversarial data 81.25
difference in accuracy without hessian 10.9375


accuracy with hessian on adversarial data 79.6875
difference in accuracy with hessian 2.34375




**Plotting the decision boundary**

In [ ]:
from collections import Counter
import random
print(X.shape)
print(X[2].shape)

m=np.random.randint(0,X.shape[0])
print(m)

value= X[333]
#print(value)
h=150
n=h**2
values=np.array([value for i in range(n)])
print(value.shape)
def plot_dec_bound(model,axis1,axis2,value,is_reg):
    cmap='Paired'
    cmap = plt.get_cmap(cmap)
    plt.figure()
      # step size in the mesh
    #create a mesh to plot in
    xx, yy = np.meshgrid(np.linspace(-10, 10, h),
                       np.linspace(-10,10, h))
    data1 = np.c_[xx.ravel(), yy.ravel()]
    data=np.reshape(data1,(data1.shape[0],data1.shape[1],1))
    meshgrid =[]
    value[:,:,axis1]=data[:,0,:]
    value[:,:,axis2]=data[:,1,:]
    values = torch.from_numpy(value)
    print("values shape",values.shape)
    if torch.cuda.is_available:
      values = values.cuda()
    outputs = model(values.float())
    #print("output has shape",outputs.shape)
    
    Z=torch.max(outputs.data, 1)[1].cpu()
    Z = Z.numpy()

    
    print("unique values of Z are: ",len(Counter(Z)))
  
    shapez=Z.shape
    #print("xx shape:",xx.shape)
    Z = Z.reshape(xx.shape)
    im=plt.contourf(xx,yy,Z,cmap="cool",alpha = 1)
    plt.xlabel("axes:%i"%axis1)
    plt.ylabel("axes:%i"%axis2)
    plt.title("regularized?:"+("true" if is_reg else "flase"))
    plt.colorbar(ticks=Z)
    if is_reg:
        plt.savefig("rnn_hess_%i.jpg"%i)
    else:
        plt.savefig("rnn_%i.jpg"%i)
    plt.show()

np.random.seed(19)
for i in range(4):
    axis1,axis2=np.random.randint(0,values.shape[2]-1,2)
    print("axes are",axis1,axis2)
    print("dec bound without jacobi")
    plot_dec_bound(model_NH,axis1,axis2,values,is_reg=False)
    print("dec bound with jacobi")
    plot_dec_bound(model_H,axis1,axis2,values,is_reg=True)


    

   
    